In [91]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
import csv
import time
from lxml.html import fromstring

In [2]:
#us cities list with coordinates
us=pd.read_csv('uscitiesv1.4.csv')

In [3]:
us.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


In [78]:
#due to the computation power, only run for 500 most populated cities in US
us_test= us.sort_values(['population'], ascending=[False])[0:100]
lat= list(us_test['lat'])
lng= list(us_test['lng'])

In [79]:
us_test.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
12359,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,19164071.0,8537673.0,10934.0,polygon,True,America/New_York,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
16410,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1140,-118.4068,12740381.0,3976322.0,3275.0,polygon,True,America/Los_Angeles,90291 90293 90292 90029 91316 91311 90035 9000...,1840020491
20044,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6861,8639278.0,2704958.0,4585.0,polygon,True,America/Chicago,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3640,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2102,6247425.0,453579.0,4866.0,polygon,True,America/New_York,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
7504,Dallas,Dallas,TX,Texas,48113,Dallas,32.7938,-96.7659,5634307.0,1317929.0,1490.0,polygon,True,America/Chicago,75287 75098 75233 75231 75234 75254 75251 7525...,1840019440


In [92]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [116]:
#function to use latitude and longitude to search for theatre locations on imax website
#input: latitude, longitude; return: name, address
def get_data(lat, lng):
    date = '2019-03-09'
    proxies = get_proxies()
    res= requests.get('https://www.imax.com/showtimes/ajax/theatres?date=' + date + '&lat=' + str(lat) + '&lon=' + str(lng),
                     proxies=proxies)
    soup= bs(res.content, 'lxml')
    newData = json.loads(soup.select_one('p').text)
    columns = ['name','address']
    baseURL = 'https://www.imax.com'
    results = []
    for row in newData['rows']:
        soup = bs(row['row'], 'lxml')
        name = soup.select_one('.theatre-title').text.strip()
        address = soup.select_one('.theatre-address').text.strip()
        results.append([name,address])
    return results

In [118]:
#get data for all the cities
results=[]
columns = ['name','address']
for i in range(len(us_test)):
    results.extend(get_data(lat[i], lng[i]))
    time.sleep(0.5)
df_100 = pd.DataFrame(results, columns = columns)

In [121]:
#save results to csv
df_100.to_csv("us100_full.csv", index=False)

In [28]:
df_test2[['address','city','state','other1','other2']] = df_test2.address.str.split(", ",expand=True)

/Users/ran/anaconda2/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [36]:
df_test2

,name,address,city,state,other1,other2
43,Edwards Camarillo Palace Stadium 12 & IMAX,680 Ventura Boulevard,Camarillo,CA 93010,None,None
44,Cinemark 22 & IMAX,2600 West Avenue I,Lancaster,CA 93534,None,None
47,Navy Pier IMAX at AMC,700 East Grand Avenue - Suite 115,Chicago,IL 60611,None,None
48,Regal City North Stadium 14 & IMAX,2600 N. Western Ave,Chicago,IL 60647,None,None
49,AMC Quarry 14 & IMAX,9201 W. 63rd Street,Hodgkins,IL 60525,None,None
50,AMC Showplace Village Crossing & IMAX,7000 Carpenter Road,Skokie,IL 60077,None,None
51,AMC Oakbrook 12 & IMAX,600 Oakbrook Center,Oak Brook,Illinois 60523,None,None
52,AMC DINE-IN Yorktown 18 & IMAX,80 Yorktown Shopping Center,Lombard,IL 60148,None,None
53,AMC Showplace Niles 12 & IMAX,301 Golf Mill Center,Niles,IL 60714,None,None
54,Cinemark at Seven Bridges & IMAX,6500 Route 53,Woodridge,IL 60517,None,None


In [46]:
max(df_test2.groupby('city').city.count())

4

In [58]:
#since NY has the largest population, calculated the number of theatres in New York City
#get data for all the cities
lat= list(us_test['lat'])
lng= list(us_test['lng'])
results_ny=get_data(lat[0], lng[0])
df_ny = pd.DataFrame(results_ny, columns = columns)
len(df_ny)

24